In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['axes.edgecolor'] = 'k'
sns.set_style('whitegrid')

In [8]:
from sklearn.linear_model import LinearRegression

def rolling_trend_line(data, window):
    trend_line = np.empty(len(data))
    trend_line[:] = np.nan
    
    for i in range(len(data) - window + 1):
        x = np.arange(i, i + window).reshape(-1, 1)
        y = data[i:i + window]
        
        model = LinearRegression()
        model.fit(x, y)
        y_pred = model.predict(x)
        
        trend_line[i:i + window] = y_pred
    
    return trend_line

In [5]:
stock = 'NVDA'
start_date = '2022-01-01'
end_date = '2023-08-27'

data = yf.download(stock, start_date, end_date).drop('Close', axis=1)
input_vars = ['Open', 'High', 'Volume']
target_var = ['Adj Close']

input = data[input_vars]
target = data[target_var]

[*********************100%***********************]  1 of 1 completed


In [6]:
input.head(3)

,Open,High,Volume
Date,,,
2022-01-03,298.149994,307.109985,39154700
2022-01-04,302.769989,304.679993,52715400
2022-01-05,289.489990,294.160004,49806400


In [7]:
target.head(3)

,Adj Close
Date,
2022-01-03,300.846375
2022-01-04,292.546417
2022-01-05,275.706818


In [ ]:
features = ['Volatility', 'Moving Average', '']